In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(1)

FS = 40            # sampling rate
WINDOW = 120       # samples (3 seconds)
N_NORMAL = 300
N_ADL = 300
N_FALL = 200

def simulate_normal():
    # Very mild random “noise” around gravity = 1g on z
    ax = 0.02*np.random.randn(WINDOW)
    ay = 0.02*np.random.randn(WINDOW)
    az = 1.0 + 0.02*np.random.randn(WINDOW)
    return ax, ay, az

def simulate_adl():
    # Moderate oscillations (e.g. walking)
    t = np.linspace(0, 3, WINDOW)
    ax = 0.15*np.sin(2*np.pi*1*t) + 0.1*np.random.randn(WINDOW)
    ay = 0.12*np.sin(2*np.pi*1.5*t + 0.8) + 0.1*np.random.randn(WINDOW)
    az = 1.0 + 0.15*np.sin(2*np.pi*1.2*t) + 0.1*np.random.randn(WINDOW)
    return ax, ay, az

def simulate_fall():
    # 1) Prefall: small motion
    ax = 0.05*np.random.randn(WINDOW)
    ay = 0.05*np.random.randn(WINDOW)
    az = 1.0 + 0.05*np.random.randn(WINDOW)

    # 2) Impact: one big spike (reported in Fig.2)
    impact_i = np.random.randint(40, 70)
    spike = np.random.uniform(2.5, 5.0)

    ax[impact_i] += spike + 0.2*np.random.randn()
    ay[impact_i] += spike + 0.2*np.random.randn()
    az[impact_i] += spike + 0.2*np.random.randn()

    # 3) Body adjustment: vibration for ~0.5 sec
    for i in range(impact_i+1, impact_i+20):
        if i < WINDOW:
            ax[i] += 0.3*np.random.randn()
            ay[i] += 0.3*np.random.randn()
            az[i] += 0.3*np.random.randn()

    # 4) Postfall: almost flat signal, but gravity lies on different axis (Fig.3)
    g_axis = np.random.choice(['x','y','z'])
    if impact_i+20 < WINDOW:
        if g_axis == 'x':
            ax[impact_i+20:] += 1.0
        elif g_axis == 'y':
            ay[impact_i+20:] += 1.0
        else:
            az[impact_i+20:] += 1.0

    return ax, ay, az


# =======================
# Build dataset
# =======================
records = []

def add_samples(count, generator, label):
    for _ in range(count):
        ax, ay, az = generator()
        for j in range(WINDOW):
            records.append([ax[j], ay[j], az[j], label])

add_samples(N_NORMAL, simulate_normal, 0)
add_samples(N_ADL, simulate_adl, 0)
add_samples(N_FALL, simulate_fall, 1)

df = pd.DataFrame(records, columns=["ax","ay","az","label"])
df.to_csv("fake_fall_data.csv", index=False)

print("Dataset saved! Shape:", df.shape)
df.head()


Dataset saved! Shape: (96000, 4)


,ax,ay,az,label
0,0.032487,-0.000492,0.990036,0
1,-0.012235,-0.015503,0.993780,0
2,-0.010563,0.025475,0.999962,0
3,-0.021459,0.039342,0.972068,0
4,0.017308,-0.037160,0.982774,0


## Load dataset and extract features

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("fake_fall_data.csv")
df.head()


,ax,ay,az,label
0,0.032487,-0.000492,0.990036,0
1,-0.012235,-0.015503,0.993780,0
2,-0.010563,0.025475,0.999962,0
3,-0.021459,0.039342,0.972068,0
4,0.017308,-0.037160,0.982774,0


In [3]:
import numpy as np

FS = 40
WINDOW = 120
HALF = WINDOW // 2

def extract_two_segment_features(ax, ay, az):
    feats = []
    for a in [ax, ay, az]:
        left = a[:HALF] #χωρισμός window se dyo merh
        right = a[HALF:]
        feats.extend([left.std(), left.mean(), right.std(), right.mean()])
    return np.array(feats)


In [4]:
X = []
y = []

for i in range(0, len(df), WINDOW):
    window = df.iloc[i:i+WINDOW]
    if len(window) < WINDOW:
        continue

    ax = window["ax"].values
    ay = window["ay"].values
    az = window["az"].values
    label = window["label"].values[0]  # all 120 have same label

    f = extract_two_segment_features(ax, ay, az)

    X.append(f)
    y.append(label)

X = np.array(X)
y = np.array(y)

print("Feature shape:", X.shape)   # (num_windows, 12)
print("Labels shape:", y.shape)


Feature shape: (800, 12)
Labels shape: (800,)


In [5]:
from sklearn.preprocessing import StandardScaler
###normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# SVM TRAIN ( 1st step - liner kernel )

In [6]:
from sklearn.svm import SVC

svm1 = SVC(kernel="linear")
svm1.fit(X_scaled, y)


SVC(kernel='linear')

In [7]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

svm2 = SVC(kernel="linear")
svm2.fit(X_poly, y)

print("Expanded feature size:", X_poly.shape[1])


Expanded feature size: 90


In [8]:
import joblib
joblib.dump((scaler, poly, svm1, svm2), "svm_models.pkl")


['svm_models.pkl']

In [9]:
W1 = svm1.coef_[0]
B1 = svm1.intercept_[0]

W2 = svm2.coef_[0]
B2 = svm2.intercept_[0]

means = scaler.mean_
stds  = scaler.scale_

with open("svm_model.h", "w") as f:
    f.write("#pragma once\n\n")

    f.write(f"#define SVM_INPUTS {len(W1)}\n")
    f.write(f"#define SVM2_INPUTS {len(W2)}\n\n")

    f.write("static const float SCALER_MEAN[SVM_INPUTS] = {")
    f.write(", ".join(map(str, means)))
    f.write("};\n")

    f.write("static const float SCALER_STD[SVM_INPUTS] = {")
    f.write(", ".join(map(str, stds)))
    f.write("};\n\n")

    f.write("static const float SVM1_W[SVM_INPUTS] = {")
    f.write(", ".join(map(str, W1)))
    f.write("};\n")
    f.write(f"static const float SVM1_B = {B1};\n\n")

    f.write("static const float SVM2_W[SVM2_INPUTS] = {")
    f.write(", ".join(map(str, W2)))
    f.write("};\n")
    f.write(f"static const float SVM2_B = {B2};\n")


In [10]:
from sklearn.metrics import classification_report

y_pred1 = svm1.predict(X_scaled)
print("SVML (linear) report:")
print(classification_report(y, y_pred1))

X_poly = poly.transform(X_scaled)
y_pred2 = svm2.predict(X_poly)
print("SVMQ (quadratic) report:")
print(classification_report(y, y_pred2))


SVML (linear) report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       600
           1       1.00      1.00      1.00       200

    accuracy                           1.00       800
   macro avg       1.00      1.00      1.00       800
weighted avg       1.00      1.00      1.00       800

SVMQ (quadratic) report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       600
           1       1.00      1.00      1.00       200

    accuracy                           1.00       800
   macro avg       1.00      1.00      1.00       800
weighted avg       1.00      1.00      1.00       800

